# Generate Data 

In [ ]:
!mkdir 'speck_32_64'

In [ ]:
#### CONFIGURATION ######

number_of_samples = 300_000

#########################

import speck_k64_p32_o32_r22 as data_generator
import numpy as np

dataset=data_generator.generate_avalanche_dataset(int(number_of_samples))
for r in range(len(dataset)):
    np.savez_compressed(f"speck_32_64/round{r}_sequences300k.npy", dataset=dataset[r])

# Run `raydist` Test 

In [ ]:
import toml
from raydist.filemanager import FileManager

for r in [0,1,2,3,4,5,6,7,8,21]:
    
    savepath = f'nc_speck32_roundid{r}'
    !mkdir '{savepath}'

    F = FileManager(savepath)
    
    datapath = f'speck_32_64/round{r}_sequences300k.npy'
    
    cfgdict = {'datapath': datapath,
                 'N_BITS': 1024,
                 'N_GPUS': 1,
                 'N_ACTORS_PER_GPU': 6,
                 'NUM_GPUS': 0.15,
                 'NUM_CPUS': 5,
                 'MODEL_ID': 'gohr_generalized',
                 'MODEL_STRENGTH': 1,
                 'MODEL_INPUTS': 1024,
                 'MODEL_OUTPUTS': 63,
                 'N_EPOCHS': 5,
                 'N_TRAIN': 147456,
                 'N_VAL': 145512,
                 'BATCHSIZE': 4096,
                 'EARLY_STOPPING': True,
                 'MAKE_NEW_FILTERS': True,
                 'N_FILTERS': 100,
                 'N_INPUT_FILTER_ELEMENTS': 961,
                 'FILTER_STRATEGY': 'random',
                 'TARGET_BITS': [],
                 'DATA_STRATEGY': 'zero'}
                 
    with open(F.filename_cfg(), 'w') as configfile:
        toml.dump(cfgdict, configfile)
        
    print("="*len(datapath))
    print(datapath)
        
    !python -m raydist.run --savepath '{savepath}'